# Metadata: Prepare Dataverse Inventory


## About
- This script prepares the volume: `Trade statistics of the treaty ports, for the period 1863-1872` for manual metadata application. 
    - **HOLLIS:** https://id.lib.harvard.edu/alma/990058255570203941/catalog
    - **DRS:** https://iiif.lib.harvard.edu/manifests/view/drs:44319007$1i
- Note that this script is tailored specifically for use with the Trade Statistics volume and is not suitable for generalization.
- **Created:** 2023/03/20
- **Updated:** 2023/03/28

## Globals

In [205]:
# path to output file
g_output_file = '../tmp/dataverse_inventory.csv'

# gather some information keyed on DRS id when we the various dataframes
g_drs_urls = {}
g_drs_image_osn = {}
g_drs_csv_osn = {}
g_drs_txt_osn = {}
g_drs_alto_osn = {}
g_drs_handwriting = {}
g_drs_two_page = {}
g_drs_table_type = {}
g_drs_table_title = {}
g_drs_multilevel_columns = {}
g_drs_multilevel_rows = {}
g_drs_computation_ready = {}
g_drs_tablegroup = {}
g_drs_tablegroup_members = {}
g_drs_csv_size = {}
g_drs_csv_shape = {}
g_drs_csv_columns = {}
g_drs_series_type = {}
g_drs_series_num = {}

# metadata associated with the entire trade statistics volume
g_volume_metadata = {
    'title':'Trade statistics of the treaty ports, for the period 1863-1872',
    'attribution':'Compiled for the Austro-Hungarian Universal Exhibition, Vienna, 1873 to illustrate the international exchange of products ; published by order of the Inspector General of Chinese Maritime Customs.',
    'author':'China. Hai guan zong shui wu si shu.',
    'published':'Shanghai : Imperial Maritime Customs, 1873.',
    'notes':'Includes separately paged documents entitled Statistics of trade, with separate title page, for the following ports: Newchwang, Tientsin, Chefoo, Hankow, Kiukiang, Chinkiang, Shanghai, Ningpo, Foochow, Tamsui, Takow, Amoy, Swatow, Canton.; Robert Hart, Inspector General.; Tables include: I. Tonnage.--II. Values.--III. Articles.--IV. Revenue.--V. Population.',
    'subjects':['Vienna International Exhibition (1873)',
                'China -- Catalogs','Harbors -- China',
                'China -- Population -- Statistics',
                'China -- Commerce -- Statistics'],
    'creation_date':'1873',
    'record_id':'990058255570203000',
    'permalink':'https://hollis.harvard.edu/permalink/f/hg18ek/01HVD_ALMA211970791270003941'
}

## Modules

In [206]:
import os
import pandas as pd
import pprint as pprint

## Prepare Content

### Read content and set up dataverse inventory `DataFrame` 

In [207]:
#
# load required files
#

# load full vendor inventory
vendor_inventory_df = pd.read_csv('../tmp/mapped_vendor_inventory.csv',index_col=None)

# load manually reviewed qc file 
manual_qc_df = pd.read_csv('../tmp/manual_file_qc.csv',index_col=None)

# load manually updated metadata file (metadata about csv files)
manual_metadata_df = pd.read_csv('../tmp/manual_metadata.csv',index_col=None)
manual_metadata_df.fillna('',inplace=True)

# load lookup table relating DRS ids, IIIF urls, and images
do_inventory_df = pd.read_csv('../tmp/do_inventory.csv')

# load NLP entities
nlp_entities_df = pd.read_csv('../tmp/nlp_entities.csv')

#
# create new inventory dataframe
#

# define columns
columns = ['drs_id','url','filename_osn', 'filepath_osn', 'file_type',
           'table_title','table_type','multilevel_columns','multilevel_rows','computation_ready',
           'table_group','table_group_members','shape','size','columns','series_type','series_num','series_name',
           'image_handwriting','image_two_page','related_image','related_csv','related_txt','related_alto','entities']

# scope notes for the new columns
column_scope_notes = {
    'drs_id':'DRS id associated with the file',
    'url':'IIIF url to resource in DRS',
    'filename_osn':'Name of file mapped to owner-supplied name',
    'filepath_osn':'Full path to file using owner-supplied name',
    'file_type':'One of: image, txt, alto, or csv',
    'table_title':'Title of the csv table, if file_type = csv',
    'table_type':'One of Categorical, Comparison, Empty, Summary, Missing, or Other',
    'multilevel_columns':'True or False',
    'multilevel_rows':'True or False',
    'computation_ready':'True or False',
    'table_group':'True or False',
    'table_group_members':'List of filenames of related tables, if any',
    'shape':'Array: [width, height]',
    'size':'Total number of cells in the table, width x height',
    'columns':'List of column names, if available',
    'series_type':'Type of series, e.g., NA, Tonnage, Values, or Article',
    'series_num':'Number of the specific series',
    'series_name':'Name of the series (series_type + series_num)',
    'image_handwriting':'True or False',
    'image_two_page':'True or False',
    'related_image':'Filename for related image',
    'related_csv':'CSV file/table related to this file, if any',
    'related_txt':'Filename for related text',
    'related_alto':'Filename for related alto file',
    'entities':'NLP entities associated with the file, if any'
}

# create new dataframe with specififed columns
dataverse_inventory_df = pd.DataFrame(columns=columns)

### Populate dataverse inventory `DataFrame`

#### Collect information from the do_inventory `DataFrame`
- Note: the `do_inventory` is extracted from the digital object's IIIF manifest
- Here, I collect the urls associated with each image in the digital object

In [208]:
# process the do_inventory_df
for row in do_inventory_df.iterrows():
    # collect information from dataframe
    drs_id = row[1].get('drs_id')
    url = row[1].get('url')
    g_drs_urls[drs_id] = url

#### Collect information from the vendor_inventory `DataFrame`
- Here, I collect the `filename_osn` and `filepath_osn` for each file in the inventory and group them by `file_type` (e.g. image, csv, alto, txt). These elements will be used later on when assigning `related_image`, `related_csv`, and `related_tx`t` fields in the final dataverse inventory

In [209]:
# process the vendor_inventory_df
for row in vendor_inventory_df.iterrows():
    # collect information from dataframe
    drs_id = row[1].get('drs_id')
    file_type = row[1].get('file_type')
    filename_osn = row[1].get('filename_osn')
    filepath_osn = row[1].get('filepath_osn')
    if (file_type == 'alto'):
        g_drs_alto_osn[drs_id] = filename_osn
    elif (file_type == 'csv'): 
        g_drs_csv_osn[drs_id] = filename_osn
    elif (file_type == 'image'): 
        g_drs_image_osn[drs_id] = filename_osn
    elif (file_type == 'txt'): 
        g_drs_txt_osn[drs_id] = filename_osn

#### Collect information based upon the DRS ID from the manual_qc inventory
- The `manual_qc_df` contains information about which images feature two-page tables (`g_drs_two_page`) and which feature handwriting (`g_drs_handwriting`)

In [210]:
# process the manual_qc_df
for row in manual_qc_df.iterrows():
    # collect variables
    drs_id = row[1].get('drs_id')
    note = row[1].get('note')
    # set defaults
    g_drs_two_page[drs_id] = False
    g_drs_handwriting[drs_id] = False
    # set values
    if note == '2-page tables':
        g_drs_two_page[drs_id] = True
        two_page = True
    elif note == 'handwriting':
        g_drs_handwriting[drs_id] = True

#### Collection information based upon the DRS ID from the manual_metadata inventory
- The `manual_metadata_df` contains information about tables that have been manually assigned, such as the `g_drs_table_type` and `g_drs_tablegroups`.

In [211]:
# process the manual_metadata_df
for row in manual_metadata_df.iterrows():
    # collect variables
    drs_id = row[1].get('drs_id')
    g_drs_table_type[drs_id] = row[1].get('table_type')
    g_drs_table_title[drs_id] = row[1].get('title')
    g_drs_series_type[drs_id] = row[1].get('table_series_type')
    g_drs_series_num[drs_id] = row[1].get('table_series_num')
    mlc = True
    if (row[1].get('multilevel_columns') == 'No'):
        mlc = False
    mlr = True
    if (row[1].get('multilevel_rows') == 'No'):
        mlr = False
    g_drs_multilevel_columns[drs_id] = mlc
    g_drs_multilevel_rows[drs_id] = mlr
    g_drs_computation_ready[drs_id] = row[1].get('computation_ready')
    g_drs_tablegroup[drs_id] = row[1].get('table_group')
    g_drs_tablegroup_members[drs_id] = row[1].get('table_group_members')

#### Populate the dataverse_inventory `DataFrame`

First, populate `dataverse_inventory_df` with contents of the mapped inventory

In [212]:
# populate with contents of the mapped inventory
for row in vendor_inventory_df.iterrows():
    drs_id = row[1].get('drs_id')
    url = ''
    filename_osn = row[1].get('filename_osn')
    filepath_osn = row[1].get('filepath_osn')
    file_type = row[1].get('file_type')
    table_title = None
    table_type = None
    multilevel_columns = None
    multilevel_rows = None
    computation_ready = False
    table_group = False
    table_group_members = []
    shape = pd.NA
    size = pd.NA
    columns = []
    series_type = None
    series_num = None
    series_name = None
    image_handwriting = False
    image_two_page = False
    related_image = None
    related_csv = None
    related_txt = None
    related_alto = None
    entities = []

    # add row to dataverse inventory
    dataverse_inventory_df.loc[len(dataverse_inventory_df.index)] = [drs_id,url,filename_osn, filepath_osn, file_type,
                                                                     table_title,table_type,multilevel_columns,multilevel_rows,computation_ready,
                                                                     table_group,table_group_members,shape,size,columns,series_type,series_num,series_name,
                                                                     image_handwriting,image_two_page,related_image,related_csv,related_txt,related_alto,
                                                                     entities]


Collect information about the shape and size for each table. Also collect column information for non-multilevel column tables

In [213]:
# directory to find actual datafiels
dir = '../data/trade_statistics_renamed/'

g_csv_errors = {}

# get just csv files from the file
csv_df = dataverse_inventory_df.loc[dataverse_inventory_df['file_type'] == 'csv']
for row in csv_df.iterrows():
    drs_id = row[1].get('drs_id')
    filepath = dir + row[1].get('filepath_osn')
    g_csv_errors[filepath] = {}
    g_drs_csv_columns[drs_id] = []
    if (os.path.exists(filepath)):
        try: 
            df = pd.read_csv(filepath,index_col=None)
            g_drs_csv_size[drs_id] = df.size
            g_drs_csv_shape[drs_id] = df.shape
            if(g_drs_multilevel_columns.get(drs_id) == False):
                g_drs_csv_columns[drs_id] =list(df.columns)
            g_csv_errors[filepath]['status'] = True
            g_csv_errors[filepath]['message'] = 'Success'
        except:
            g_csv_errors[filepath]['status'] = False
            g_csv_errors[filepath]['message'] = 'Failed to read CSV'
    else:
            g_csv_errors[filepath]['status'] = False
            g_csv_errors[filepath]['message'] = 'File Not Found'

Next, based upon `file_type`, populate row values with contents from dictionaries of information collected from each of the other `DataFrame`s

In [214]:
# traverse the rows, updating values
for row in dataverse_inventory_df.iterrows():
    # get row data
    drs_id = row[1].get('drs_id')
    file_type = row[1].get('file_type')

    # set row variables
    url = g_drs_urls.get(drs_id)
    handwriting = g_drs_handwriting.get(drs_id)
    two_page = g_drs_handwriting.get(drs_id)
    related_image = g_drs_image_osn.get(drs_id)
    related_csv = g_drs_csv_osn.get(drs_id)
    related_txt = g_drs_txt_osn.get(drs_id)
    related_alto = g_drs_alto_osn.get(drs_id)
    table_type = g_drs_table_type.get(drs_id)
    table_title = g_drs_table_title.get(drs_id)
    multilevel_columns = g_drs_multilevel_columns.get(drs_id)
    multilevel_rows = g_drs_multilevel_rows.get(drs_id)
    computation_ready = g_drs_computation_ready.get(drs_id)
    table_group = g_drs_tablegroup.get(drs_id)
    table_group_members = g_drs_tablegroup_members.get(drs_id)
    size = g_drs_csv_size.get(drs_id)
    shape = g_drs_csv_shape.get(drs_id)
    columns = g_drs_csv_columns.get(drs_id)
    series_type = g_drs_series_type.get(drs_id)
    series_num = g_drs_series_num.get(drs_id)
    series_name = ''
    if ((series_type is None) or 
        (series_type == '') or
        (series_type is pd.NA) or
        (series_num is None) or
        (series_num == '') or
        (series_num is pd.NA)):
        series_name = 'Miscellaneous'
    else:
        series_name = '{}:{:d}'.format(series_type, int(series_num))

    # update row
    index = row[0]
    dataverse_inventory_df.at[index,'url'] = url
    dataverse_inventory_df.at[index,'image_handwriting'] = handwriting
    dataverse_inventory_df.at[index,'image_two_page'] = two_page
    dataverse_inventory_df.at[index,'related_image'] = related_image
    dataverse_inventory_df.at[index,'related_txt'] = related_txt
    dataverse_inventory_df.at[index,'related_csv'] = related_csv    
    dataverse_inventory_df.at[index,'table_type'] = table_type
    dataverse_inventory_df.at[index,'table_title'] = table_title
    dataverse_inventory_df.at[index,'multilevel_columns'] = multilevel_columns  
    dataverse_inventory_df.at[index,'multilevel_rows'] = multilevel_rows
    dataverse_inventory_df.at[index,'computation_ready'] = computation_ready
    dataverse_inventory_df.at[index,'table_group'] = table_group
    dataverse_inventory_df.at[index,'table_group_members'] = table_group_members
    dataverse_inventory_df.at[index,'size'] = size
    dataverse_inventory_df.at[index,'shape'] = shape
    dataverse_inventory_df.at[index,'columns'] = columns
    dataverse_inventory_df.at[index,'series_type'] = series_type
    dataverse_inventory_df.at[index,'series_num'] = series_num
    dataverse_inventory_df.at[index,'series_name'] = series_name


Process the NLP entities file: find the matching `filename_osn` from the `vendor_inventory_df`

In [216]:
# add column for filename_osn
nlp_entities_df['filename_osn'] = ''

# for each nlp entity, find the corresponding filename_osn
for row in nlp_entities_df.iterrows():
    filename = row[1].get('filename')
    df = vendor_inventory_df.loc[vendor_inventory_df['filename'] == filename]
    if (df.empty == False):
        df_index = df.index[0]
        filename_osn = df.at[df_index,'filename_osn']
        index = row[0]
        nlp_entities_df.at[index,'filename_osn'] = filename_osn
    else:
        print('Warning: not found in vendor inventory: {}'.format(filename))

# add the entities to the correct filename_osn in the dataverse_inventory
for row in nlp_entities_df.iterrows():
    filename_osn = row[1].get('filename_osn')
    entities = row[1].get('entities')
    # update matching rows in dataverse_inventory_df with entities
    dataverse_inventory_df.loc[dataverse_inventory_df['filename_osn'] == filename_osn, 'entities'] = entities

Add bibliographic metadata to the `dataverse_inventory_df`

In [217]:
dataverse_inventory_df['volume_title'] = g_volume_metadata.get('title')
dataverse_inventory_df['attribution'] = g_volume_metadata.get('attribution')
dataverse_inventory_df['author'] = g_volume_metadata.get('author')
dataverse_inventory_df['published'] = g_volume_metadata.get('published')
dataverse_inventory_df['notes'] = g_volume_metadata.get('notes')
dataverse_inventory_df['subjects'] = ';'.join(g_volume_metadata.get('subjects'))
dataverse_inventory_df['creation_date'] = g_volume_metadata.get('creation_date')
dataverse_inventory_df['record_id'] = g_volume_metadata.get('record_id')
dataverse_inventory_df['permalink'] = g_volume_metadata.get('permalink')

#### Save `dataverse_inventory_df`

In [218]:
dataverse_inventory_df.to_csv(g_output_file,index=None)

**End document.**